<a href="https://colab.research.google.com/github/mohsenpartovi73/CNN_PRSM_1/blob/main/vegetables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

kagglehub.login()

import kagglehub

# Download latest version
path = kagglehub.dataset_download("misrakahmed/vegetable-image-dataset")

print("Path to dataset files:", path)


100%|██████████| 534M/534M [00:05<00:00, 105MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/misrakahmed/vegetable-image-dataset/versions/1
Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
import torch.utils.data.dataloader
from torch.utils.data import Dataset,DataLoader
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2 as cv
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import pandas as pd
from PIL import Image
from torch.optim import lr_scheduler
import os
import random

In [ ]:
Train_path = r'/root/.cache/kagglehub/datasets/misrakahmed/vegetable-image-dataset/versions/1/Vegetable Images/train'
Test_path = r'/root/.cache/kagglehub/datasets/misrakahmed/vegetable-image-dataset/versions/1/Vegetable Images/test'
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()
                               ,transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)) ])

In [ ]:
Train_dataset = torchvision.datasets.ImageFolder(root=Train_path,transform=transform)
Test_dataset = torchvision.datasets.ImageFolder(root=Test_path,transform=transform)
Train_loader = torch.utils.data.DataLoader(dataset=Train_dataset,batch_size=64,shuffle=False)
Test_loader = torch.utils.data.DataLoader(dataset=Test_dataset,batch_size=64,shuffle=True)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(8*8*32,1024)
        self.fc2 = nn.Linear(1024,15)
        # self.fc3 = nn.Linear(512,15)


    def forward(self,x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        # x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x


In [ ]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
num_epoch = 50
for epoch in range(num_epoch):
  for i ,(images,labels) in enumerate(Train_loader):
    outputs= model(images)
    loss = criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 235 == 0:
            print (f'Epoch [{epoch+1}/{50}], Step [{i+1}/ {len(Train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/50], Step [235/ 235], Loss: 2.7069
Epoch [2/50], Step [235/ 235], Loss: 2.7163
Epoch [3/50], Step [235/ 235], Loss: 2.3260
Epoch [4/50], Step [235/ 235], Loss: 1.8808
Epoch [5/50], Step [235/ 235], Loss: 0.3329
Epoch [6/50], Step [235/ 235], Loss: 0.1666
Epoch [7/50], Step [235/ 235], Loss: 0.0784
Epoch [8/50], Step [235/ 235], Loss: 1.3923
Epoch [9/50], Step [235/ 235], Loss: 0.3260
Epoch [10/50], Step [235/ 235], Loss: 0.4008
Epoch [11/50], Step [235/ 235], Loss: 0.1671
Epoch [12/50], Step [235/ 235], Loss: 0.2381
Epoch [13/50], Step [235/ 235], Loss: 0.2007
Epoch [14/50], Step [235/ 235], Loss: 0.1239
Epoch [15/50], Step [235/ 235], Loss: 0.0942
Epoch [16/50], Step [235/ 235], Loss: 0.0790
Epoch [17/50], Step [235/ 235], Loss: 0.0501
Epoch [18/50], Step [235/ 235], Loss: 0.0412
Epoch [19/50], Step [235/ 235], Loss: 0.0455
Epoch [20/50], Step [235/ 235], Loss: 0.0474
Epoch [21/50], Step [235/ 235], Loss: 0.0339
Epoch [22/50], Step [235/ 235], Loss: 0.0442
Epoch [23/50], Step

In [ ]:
# Saving and Testing the Model
# Save the trained model
torch.save(model.state_dict(), '/content/vget_model/vegetable.pt')
print("Model saved.")


Model saved.


In [ ]:
# Load the saved model
loaded_model = CNN()
loaded_model.load_state_dict(torch.load('/content/vget_model/vegetable.pt'))
loaded_model = loaded_model
loaded_model.eval()

<ipython-input-45-f8e497084ce4>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('/content/vget_model/vegetable.pt'))


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=15, bias=True)
)

For example, here we download and display a PNG image of the Colab logo: